In [0]:
%%capture
!pip install tf-nightly-gpu   # For GPU usage, neural networks
!pip install wget

In [0]:
import tensorflow as tf
from google.colab import files
import os
import wget
import torchvision.transforms as transforms
from PIL import Image, ImageFile
import numpy as np
app = tf.keras.applications
layers = tf.keras.layers

In [0]:
def QualityNet(weights=None):

  r = app.inception_resnet_v2.InceptionResNetV2(include_top=False,
                                                weights=None,
                                                input_tensor=None, 
                                                input_shape=(224, 224, 3), 
                                                pooling='avg')
  
  dense_1 = layers.Dense(2048, activation='relu', name='2048')
  dense_2 = layers.Dense(1024, activation='relu', name='1024')
  dense_3 = layers.Dense(256, activation='relu', name='256')
  pred    = layers.Dense(1, activation='relu', name='prediction')

  model = tf.keras.Sequential([ r,
                                dense_1,
                                dense_2,
                                dense_3,
                                pred ])
  if weights is not None:
    model.load_weights(weights)
  else:
    # if not os.path.exists("model.h5"):

    filepath = "https://www.dropbox.com/s/ov20jow5zj42oi5/model.h5?dl=1"
    wget.download(filepath, "./model.h5")
    model.load_weights("model.h5")

  return model

In [0]:
model = QualityNet()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 1536)              54336736  
_________________________________________________________________
2048 (Dense)                 (None, 2048)              3147776   
_________________________________________________________________
1024 (Dense)                 (None, 1024)              2098176   
_________________________________________________________________
256 (Dense)                  (None, 256)               262400    
_________________________________________________________________
prediction (Dense)           (None, 1)                 257       
Total params: 59,845,345
Trainable params: 59,784,801
Non-trainable params: 60,544
_________________________________________________________________


In [0]:
class RemoveAlphaChannel(object):
  # This function removes the alpha channel of a picture in case
  # it posseses one. 
  # If not, nothing is done
  def __call__(self, image_tensor):
    if image_tensor.shape[0] == 4:
      image_tensor = image_tensor[:3]
    assert image_tensor.shape[0] == 3
    return image_tensor

normalize = transforms.Compose([transforms.Resize([224, 224]),
                                transforms.ToTensor(),
                                RemoveAlphaChannel(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                     std=[0.229, 0.224, 0.225]),
                              ])

def predict(image, model):
  ts = np.reshape(normalize(Image.open(image).convert('RGB')).unsqueeze(0).numpy(), (1, 224, 224, 3))
  return model.predict(img_tensor)[0][0]


In [0]:
files.upload()

image = "1998.jpg" # Change to name of uploaded image

Saving 1998.jpg to 1998.jpg


In [0]:
predict(image, model)

9.457901